In [16]:
import os
import pandas as pd
from typing import List, Any
from crewai.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from crewai import Agent, Task, Crew
from deepeval.test_case import LLMTestCase, ToolCall
from deepeval.metrics import ToolCorrectnessMetric 
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric
# 1. SETUP: Mock Data (Replacements for your MySQL/ChromaDB)
# We mock these so we can test logic without needing the live DB running
credit_data = {
    "ID": ["1111", "2222", "3333", "4444", "5555"],
    "Name": ["Loren", "Matt", "Hilda", "Andy", "Kit"],
    'Email': ['loren@test.com', 'matt@test.com', 'hilda@test.com', 'andy@test.com', 'kit@test.com'],
    "Credit_Score": [455, 685, 825, 840, 350]
}
credit_df = pd.DataFrame(credit_data)

account_data = {
    "ID": ["1111", "2222", "3333", "4444", "5555"],
    "Name": ["Loren", "Matt", "Hilda", "Andy", "Kit"],
    "Nationality": ["Singaporean", "Non-Singaporean", "Singaporean", "Non-Singaporean", "Singaporean"],
    "Account_Status": ["good-standing", "closed", "delinquent", "good-standing", "delinquent"]
}
account_df = pd.DataFrame(account_data)

# --- 3. Simulated Government PR System ---
pr_data = {
    "ID": ["2222", "4444", "1111", "3333", "5555"],
    "Name": ["Matt", "Andy", "Loren", "Hilda", "Kit"],
    "PR_Status": ["true", "false", "true", "true", "false"]
}
pr_df = pd.DataFrame(pr_data)




In [17]:
# 2. LOGGING WRAPPER (The Fix)
# Instead of a class, we use a closure. This is impossible for CrewAI to reject.
def create_tool(name: str, func, description: str, log_list: list):
    
    def wrapper(query: str):
        # Run the logic
        result = func(query)
        # Log it
        log_list.append(
            ToolCall(name=name, input=str(query), output=str(result))
        )
        return str(result)

    # Return a standard LangChain Tool using the .from_function method
    return Tool.from_function(
        func=wrapper,
        name=name,
        description=description
    )

In [18]:
# --- 1. Configuration for Embeddings ---
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=250)

# --- 2. Build Vector DB for RISK POLICY ---
print("Building Risk Policy Vector DB...")
risk_loader = PyPDFLoader("Bank Loan Overall Risk Policy.pdf")
risk_docs = risk_loader.load()
risk_splits = text_splitter.split_documents(risk_docs)

# Create the Vector Store specifically for Risk
risk_vector_db = Chroma.from_documents(
    documents=risk_splits,
    embedding=embeddings,
    collection_name="risk_policy_collection",
    persist_directory="./risk_db" # Optional: Saves to disk
)

# --- 3. Build Vector DB for INTEREST RATE POLICY ---
print("Building Interest Rate Policy Vector DB...")
interest_loader = PyPDFLoader("Bank Loan Interest Rate Policy.pdf")
interest_docs = interest_loader.load()
interest_splits = text_splitter.split_documents(interest_docs)

# Create the Vector Store specifically for Interest Rates
interest_vector_db = Chroma.from_documents(
    documents=interest_splits,
    embedding=embeddings,
    collection_name="interest_rate_collection",
    persist_directory="./interest_db" # Optional: Saves to disk
)

print("✅ Two separate Vector Databases created successfully.")

Building Risk Policy Vector DB...
Building Interest Rate Policy Vector DB...
✅ Two separate Vector Databases created successfully.


In [19]:
# 3. DEFINE TOOL LOGIC (Extracted from app.py)
# Note: We do NOT use the @tool decorator here. We wrap them manually below.
# 2. GLOBAL LOG LIST (This is where we capture the calls)
tool_usage_log = []

# 3. DEFINE TOOLS (Using the @tool decorator, same as app.py)
@tool("Get Customer ID")
def get_customer_id(query: str):
    """Fetches the unique Customer ID and Name by searching for an ID, Name, or Email."""
    
    # --- LOGIC ---
    q = query.strip().lower()
    mask = credit_df['Name'].str.lower().str.contains(q, na=False)
    results = credit_df[mask]
    
    if results.empty:
        output = "No customer found."
    else:
        row = results.iloc[0]
        output = f"ID: {row['ID']}, Name: {row['Name']}"

    # --- THE TRICK: Log it right here ---
    tool_usage_log.append(
        ToolCall(name="Get Customer ID", input=str(query), output=str(output))
    )
    return output

@tool("Get Credit Score")
def get_credit_score(customer_id: str):
    """Fetches the credit score. Requires a unique Customer ID."""
    
    # --- LOGIC ---
    row = credit_df[credit_df['ID'] == str(customer_id)]
    if row.empty: 
        output = "Score not found."
    else:
        output = str(row.iloc[0]['Credit_Score'])

    # --- LOG IT ---
    tool_usage_log.append(
        ToolCall(name="Get Credit Score", input=str(customer_id), output=str(output))
    )
    return output

@tool("Get Account Status")
def get_account_status(customer_id: str):
    """Fetches Account Status & Nationality. Requires a Customer ID."""
    
    # 1. ACTUAL LOGIC
    # Ensure string comparison for ID
    row = account_df[account_df['ID'] == str(customer_id)]
    
    if row.empty: 
        output = "Account Status not found."
    else:
        output = f"Status: {row.iloc[0]['Account_Status']}, Nationality: {row.iloc[0]['Nationality']}"

    # 2. LOGGING
    tool_usage_log.append(
        ToolCall(name="Get Account Status", input=str(customer_id), output=str(output))
    )
    return output

@tool("Get PR Status")
def get_pr_status(customer_id: str):
    """Fetches PR Status. Requires a Customer ID."""
    
    # 1. ACTUAL LOGIC
    # Ensure pr_df is available in your scope (mock or real)
    row = pr_df[pr_df['ID'] == str(customer_id)]
    
    if row.empty: 
        output = "PR Status not found."
    else:
        output = str(row.iloc[0]['PR_Status'])

    # 2. LOGGING
    tool_usage_log.append(
        ToolCall(name="Get PR Status", input=str(customer_id), output=str(output))
    )
    return output


@tool("Risk Policy Search")
def search_risk_policy(query: str):
    """Searches the 'Bank Loan Overall Risk Policy' PDF for risk rules."""
    
    # 1. ACTUAL LOGIC (Using your Vector DB)
    results = risk_vector_db.similarity_search(query, k=3)
    output = "\n".join([doc.page_content for doc in results])
    
    if not output: output = "No relevant policy found in PDF."

    # 2. LOGGING
    tool_usage_log.append(
        ToolCall(name="Risk Policy Search", input=str(query), output=str(output))
    )
    return output

@tool("Interest Rate Policy Search")
def search_interest_rate_policy(query: str):
    """Searches the 'Bank Loan Interest Rate Policy' PDF for interest rate rules."""
    
    # 1. ACTUAL LOGIC (Using your Vector DB)
    results = interest_vector_db.similarity_search(query, k=3)
    output = "\n".join([doc.page_content for doc in results])
    
    if not output: output = "No relevant policy found in PDF."

    # 2. LOGGING
    tool_usage_log.append(
        ToolCall(name="Interest Rate Policy Search", input=str(query), output=str(output))
    )
    return output


In [ ]:
# 5. RECREATE AGENTS (Using Wrapped Tools)
# Ensure you set your OpenAI Key environment variable before running!
os.environ["OPENAI_API_KEY"] = "Nil" 

llm = ChatOpenAI(model_name="gpt-4", temperature=0.1)

# Define Agents (Re-instantiated with the live LLM)
# Define Agents (Re-instantiated with the live LLM)
# Define Agents (Re-instantiated with the live LLM)
router_agent = Agent(
    role='Loan Workflow Manager',
    goal='Orchestrate the end-to-end loan application process and ensure data integrity.',
    backstory=(
        "You are the Lead Loan Officer."
        "You manage the flow of information. You must validate the output of every specialist before proceeding."
        "\n\n"
        "YOUR PROCESS:"
        "1. Ask Data Specialist for Profile."
        "   -> **CIRCUIT BREAKER**: If Data Specialist returns 'STATUS: NOT_FOUND' or 'STATUS: DATA_MISSING', STOP. "
        "      Reply: 'Error: Customer record could not be retrieved. Please verify the name/ID.'"
        "\n\n"
        "2. Check Eligibility (as per previous instructions)."
        "   -> If 'INELIGIBLE':"
        "     STOP the process. Do NOT contact Risk or Interest specialists."
        "     **Draft a Final Rejection Report:**"
        "     You must acknowledge the applicant's Credit Standing in your rejection."
        "     Format: 'Although the applicant has a [Good/Bad] Credit Score of {insert_score}, we cannot recommend this loan because {insert_rejection_reason}.'"
        "\n\n"
        "3. Ask Risk Specialist for Risk Level."
        "   -> **CIRCUIT BREAKER**: If Risk Specialist returns 'MANUAL_REVIEW_NEEDED', STOP."
        "      Reply: 'Process Halted: Application requires manual underwriting review due to outlier credit score.'"
        "\n\n"
        "4. Ask Interest Specialist for Rate (Only if Risk Level is valid)."
        "   -> **CIRCUIT BREAKER**: If Interest Specialist returns 'Undetermined', STOP."
        "\n\n"
        "5. Compile final report only if all steps succeeded."
    ),
    allow_delegation=True,
    llm=llm,
    verbose=True
)

data_agent = Agent(
    role='Data Retrieval Specialist',
    goal='Fetch and aggregate all necessary customer information using any available identifier.',
    backstory=(
        "You are an expert Banking Database Analyst."
        "You must execute the following workflow strictly in order:"
        "\n\n"
        "STEP 1: Use 'Get Customer ID' using the provided input."
        "   - **CRITICAL FAILURE CHECK**: If the tool returns 'No customer found', STOP. Return 'STATUS: NOT_FOUND'."
        "\n\n"
        "STEP 2: Use the Customer ID to fetch 'Credit Score' and 'Account Status'."
        "   - If these fields are empty/null, Return 'STATUS: DATA_MISSING'."
        "\n\n"
        "STEP 3: Check Nationality (from Account Status)."
        "   - **CRITICAL PATH**: "
        "   - IF (Nationality == 'Singaporean'): **STOP USING TOOLS IMMEDIATELY.** Proceed to Step 4 (Output)."
        "   - IF (Nationality == 'Non-Singaporean'): You MUST use the 'Get PR Status' tool."
        "\n\n"
        "STEP 4: OUTPUT (THE FINAL ANSWER):"
        "   - Do not use any more tools."
        "   - Generate your Final Answer containing the Profile Data, Eligibility Status, and Rejection Reason."
        "   - IF (Nationality == 'Non-Singaporean' AND PR Status == False): Status = 'INELIGIBLE'."
        "   - ELSE: Status = 'ELIGIBLE'."
        ),
    tools=[get_customer_id, get_credit_score, get_account_status, get_pr_status],
    llm=llm,
    verbose=True,
    allow_delegation=False
)

# 3. Risk Assessment Specialist (Unchanged)
risk_agent = Agent(
    role='Risk Assessment Specialist',
    goal='Determine the Risk Level (Low/Medium/High) based on the policy documents.',
    backstory=(
        "You are a Risk Underwriter. You work with the 'Bank Loan Overall Risk Policy'."
        "You will receive a Customer Profile containing a Credit Score and Account Status."
        "Your sole responsibility is to search the Risk Policy to find which Risk Level corresponds to that specific Credit Score and Account Status."
        "\n\n"
        "FAILURE HANDLING:"
        "If the search results do NOT contain a rule for the specific credit score provided (e.g., score is too low/high or policy is ambiguous), you MUST:"
        "1. DO NOT GUESS."
        "2. Return 'Risk Level: MANUAL_REVIEW_NEEDED'."
        "3. State the reason (e.g., 'Credit score 350 is below policy threshold')."
    ),
    tools=[search_risk_policy],
    verbose=True,
    llm=llm
)

# 4. Interest Rate Specialist (Unchanged)
interest_agent = Agent(
    role='Interest Rate Specialist',
    goal='Determine the applicable Interest Rate based on the Risk Level.',
    backstory=(
        "You are a Pricing Analyst."
        "You will receive a 'Risk Level' (e.g., Low, Medium, High) from the Risk Assessment Specialist."
        "Search the Interest Rate Policy to find the specific interest rate percentage associated with that Risk Level."
        "Return only the numerical rate and the justification."
        "\n\n"
        "FAILURE HANDLING:"
        "If the search results do NOT contain a rule for the specific credit score provided (e.g., score is too low/high or policy is ambiguous), you MUST:"
        "1. DO NOT GUESS."
        "2. Return 'Risk Level: MANUAL_REVIEW_NEEDED'."
        "3. State the reason (e.g., 'interest rate cannot be found for current credit score')."
    ),
    tools=[search_interest_rate_policy],
    verbose=True,
    llm=llm
)

In [21]:
relevancy_metric = AnswerRelevancyMetric(threshold=0.7)
faithfulness_metric = FaithfulnessMetric(threshold=0.7)
# 6. DEFINE TASK & CREW
customer_name = "loren"
user_input = f"What is the overall interest rate for customer {customer_name}?"
# expected_output = "The final Risk Level for the loan application of the customer named 'loren' is Medium. The interest rate for the loan is 4.885%."
expected_output = """Final Output: | Customer ID | Name  | Nationality | PR Status | Credit Score | Risk Level | Interest Rate |    │
│  Final Decision |                                                                                               │
│  |-------------|-------|-------------|-----------|--------------|------------|---------------|----------------  │
│  |                                                                                                              │
│  | 1111        | Loren | Singaporean | N/A       | 455          | Medium     | 4.885%        | Approved         │
│  |                                                                                                              │
│                                                                                                                 │
│  The final decision is to approve the loan for Loren. This decision is based on Loren's eligibility, medium     │
│  risk level, and the determined interest rate of 4.885%."""             

task = Task(
    description=(
        f"Process the full loan application for: {customer_name}"
        "\n\n"
        "Execution Steps:"
        "1. DATA: Retrieve profile and check Eligibility (Singaporean or PR). "
        "2. DECISION: If Ineligible, terminate with a rejection notice. "
        "3. RISK: ONLY if Eligible, proceed to determine Risk Level with credit score & account status. "
        "4.PRICING: ONLY if Risk Level is valid, determine Interest Rate. "
        "5. REPORT: Generate a final summary table. State the reason for the final decision."
    ),
    expected_output='A Markdown table containing: Customer ID, Name, Nationality, PR Status, Credit Score, Risk Level, Interest Rate, and Final Decision. Provide a sentence on the reason for the final decision.',
    agent=router_agent
)

crew = Crew(
    agents=[router_agent, data_agent, risk_agent, interest_agent],
    tasks=[task],
    verbose=True
)

print("🚀 Starting Crew Execution for Evaluation...")
crew_output = crew.kickoff()
# crew_output = crew.kickoff(inputs={"query": user_input})
actual_output = str(crew_output)

print(f"✅ Final Answer: {actual_output}\n")

🚀 Starting Crew Execution for Evaluation...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3c1265ed-5136-4670-a184-700afe7519ce                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Loan Workflow Manager                                                                                   │
│                                                                                                                 │
│  Task: Process the full loan application for: loren                                                             │
│                                                                                                                 │
│  Execution Steps:1. DATA: Retrieve profile and check Eligibility (Singaporean or PR). 2. DECISION: If           │
│  Ineligible, terminate with a rejection notice. 3. RISK: ONLY if Eligible, proceed to determine Risk Level      │
│  with credit score & account status. 4.PRICING: ONLY if Risk Level is valid, determine Interest Rate. 5.        │
│  REPORT: Generate a final summary table. State the reason for the final decision.                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\devil\Downloads\GroupProject\venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Retrieval Specialist                                                                               │
│                                                                                                                 │
│  Task: Retrieve the profile of the loan applicant                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 06d4983d-cdaf-440c-8c2b-e2839f94ffda                                                                     │
│  Agent: Loan Workflow Manager                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Loan Workflow Manager                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  | Customer ID | Name  | Nationality | PR Status | Credit Score | Risk Level | Interest Rate | Final Decision   │
│  |                                                                                                              │
│  |-------------|-------|-------------|-----------|--------------|------------|---------------|----------------  │
│  |                                                                                                              │
│  | 1111        | Loren | Singaporean | N/A       | 455          | Medium     | 4.885%        | Approved         │
│  |                                                                                                              │
│                                                                                                                 │
│  The final decision is to approve the loan for Loren. This decision is based on Loren's eligibility status,     │
│  credit score, risk level, and the determined interest rate.                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Final Answer: | Customer ID | Name  | Nationality | PR Status | Credit Score | Risk Level | Interest Rate | Final Decision |
|-------------|-------|-------------|-----------|--------------|------------|---------------|----------------|
| 1111        | Loren | Singaporean | N/A       | 455          | Medium     | 4.885%        | Approved       |

The final decision is to approve the loan for Loren. This decision is based on Loren's eligibility status, credit score, risk level, and the determined interest rate.


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3c1265ed-5136-4670-a184-700afe7519ce                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: | Customer ID | Name  | Nationality | PR Status | Credit Score | Risk Level | Interest Rate |    │
│  Final Decision |                                                                                               │
│  |-------------|-------|-------------|-----------|--------------|------------|---------------|----------------  │
│  |                                                                                                              │
│  | 1111        | Loren | Singaporean | N/A       | 455          | Medium     | 4.885%        | Approved         │
│  |                                                                                                              │
│                                                                                                                 │
│  The final decision is to approve the loan for Loren. This decision is based on Loren's eligibility status,     │
│  credit score, risk level, and the determined interest rate.                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [22]:
retrieval_context = []
for log in tool_usage_log:
    # We add the output of every tool call to the context
    # This ensures 'ID: 1111' and 'Score: 455' and 'Policy Rules' are all present.

   # retrieval_context.append(str(log.output))
    retrieval_context.append(str(log.output))
    print(retrieval_context[-1])
# 5a. EVALUATE      ## For faithfulness test case
# ------------------------------------------------------------------
test_case = LLMTestCase(
    input=customer_name,                
    actual_output=actual_output,
    expected_output=expected_output,
    retrieval_context=retrieval_context 
)
# 5b. EVALUATE      ## For relevancy test case
# ------------------------------------------------------------------
# test_case = LLMTestCase(
#     ##input="Process loan for Loren",   
#     actual_output=actual_output,
#     expected_output="Medium risk interest rate is 4.885%",
#     retrieval_context=retrieval_context 
# )
# ------------------------------------------------------------------
retrieval_context = []
for log in tool_usage_log:
    # We add the output of every tool call to the context
    # This ensures 'ID: 1111' and 'Score: 455' and 'Policy Rules' are all present.
    retrieval_context.append(str(log.output))

##print("🚀 Running Evaluation Metrics for "+customer_name+"\n")
print("🚀 Running Evaluation Metrics for Lorean")

print("🟢 Customer Name:")
print("Loren")

print("🟢 Prompt")
print(test_case.input)

print("📝 Actual Output:")
print(actual_output)

print("✅ Expected Output:")
print(expected_output)

print("📚 Retrieval Context (from tool outputs):")
for i, ctx in enumerate(retrieval_context, start=1):
    print(f"{i}. {ctx}")

# --- Measure Faithfulness ---
faithfulness_metric.measure(test_case)
print(f"Faithfulness Score: {faithfulness_metric.score}")
print(f"Reason: {faithfulness_metric.reason}\n")

# --- Measure Answer Relevancy ---
# Checks if the answer is not gibberish and addresses the question
relevancy_metric.measure(test_case)
print(f"Relevancy Score: {relevancy_metric.score}")
print(f"Reason: {relevancy_metric.reason}\n")

ID: 1111, Name: Loren
455
Status: good-standing, Nationality: Singaporean
Bank Loan Overall Risk 
The table below provides the overall risk for initiating a bank loan based on the customer’s 
credit score and account status. 
 
Credit Score Account Status Overall Risk 
300 – 674 Delinquent High 
675 - 749 Delinquent High 
750 - 850 Delinquent Medium 
300 – 674 Closed High 
675 - 749 Closed Medium 
750 - 850 Closed Low 
300 – 674 Good-standing Medium 
675 - 749 Good-standing Medium 
750 - 850 Good-standing Low
Bank Loan Overall Risk 
The table below provides the overall risk for initiating a bank loan based on the customer’s 
credit score and account status. 
 
Credit Score Account Status Overall Risk 
300 – 674 Delinquent High 
675 - 749 Delinquent High 
750 - 850 Delinquent Medium 
300 – 674 Closed High 
675 - 749 Closed Medium 
750 - 850 Closed Low 
300 – 674 Good-standing Medium 
675 - 749 Good-standing Medium 
750 - 850 Good-standing Low
Bank Loan Overall Risk 
The table below prov

ERROR:deepeval.retry.openai:call timed out after 88.5s (per attempt). Increase DEEPEVAL_PER_ATTEMPT_TIMEOUT_SECONDS_OVERRIDE (None disables) or reduce work per attempt. Retrying: 1 time(s)...
INFO:deepeval.retry.openai:Retrying in 1.404656961817902 s (attempt 1) after TimeoutError('call timed out after 88.5s (per attempt). Increase DEEPEVAL_PER_ATTEMPT_TIMEOUT_SECONDS_OVERRIDE (None disables) or reduce work per attempt.')
ERROR:deepeval.retry.openai:call timed out after 88.5s (per attempt). Increase DEEPEVAL_PER_ATTEMPT_TIMEOUT_SECONDS_OVERRIDE (None disables) or reduce work per attempt. Retrying: 2 time(s)...


RetryError: RetryError[<Future at 0x1d5a7819e50 state=finished raised TimeoutError>]

Faithfulness Score: 1.0

Reason: The score is 1.00 because there are no contradictions—great job staying true to the retrieval context!

Relevancy Score: 1.0

Reason: The score is 1.00 because the answer was fully relevant and addressed the question directly without any irrelevant information. Great job staying focused and concise!

In [ ]:
# 7. DEEPEVAL EVALUATION
print("\n------------------------------------------------")
print("📊 Starting DeepEval Assessment")
print("------------------------------------------------")

# Define what we EXPECTED the agents to do
expected_tools_sequence = [
    ToolCall(
        name="Get Customer ID",
        input="loren",
        output="ID: 1111, Name: loren"
    ),
    ToolCall(
        name="Get Credit Score",
        input="1111", 
        output="455"
    ),
    ToolCall(
        name="Get Account Status",
        input="1111", 
        output="good-standing"
    ),
    ##########################
    ToolCall(
        name="Risk Policy Search",
        input="Credit Score: 455 , account status: good-standing",
        output="Overall risk: Medium"
    ),
    ToolCall(
        name="Interest Rate Policy Search",
        input="Overall risk: Medium",
        output="Interest Rate: 4.885 %"
    )
]

# Create the Test Case
test_case = LLMTestCase(
    input=f"Process loan for Lorean",
    actual_output=str(crew_output),
    tools_called=tool_usage_log, 
    expected_tools=expected_tools_sequence
)

# Run the Metric
metric = ToolCorrectnessMetric()
metric.measure(test_case)

print(f"Prompt: {test_case.input}")
print(f"✅ Tool Correctness Score: {metric.score}")
print(f"📝 Reason: {metric.reason}")

# Print the captured log for debugging
print("\n🔧 Actual Tool Usage Log:")
for log in tool_usage_log:
    print(log)


------------------------------------------------
📊 Starting DeepEval Assessment
------------------------------------------------


Prompt: Process loan for Lorean
✅ Tool Correctness Score: 1.0
📝 Reason: [
	 Tool Calling Reason: All expected tools ['Get Customer ID', 'Get Credit Score', 'Get Account Status', 'Risk Policy Search', 'Interest Rate Policy Search'] were called (order not considered).
	 Tool Selection Reason: No available tools were provided to assess tool selection criteria
]


🔧 Actual Tool Usage Log:
name='Get Customer ID' description=None reasoning=None output='ID: 1111, Name: Loren' input_parameters=None
name='Get Credit Score' description=None reasoning=None output='455' input_parameters=None
name='Get Account Status' description=None reasoning=None output='Status: good-standing, Nationality: Singaporean' input_parameters=None
name='Risk Policy Search' description=None reasoning=None output='Bank Loan Overall Risk \nThe table below provides the overall risk for initiating a bank loan based on the customer’s \ncredit score and account status. \n \nCredit Score Account Status Overall Risk \n300 – 674

Loren

✅ Tool Correctness Score: 1.0
📝 Reason: [
	 Tool Calling Reason: All expected tools ['Get Customer ID', 'Get Credit Score'] were called (order not considered).
	 Tool Selection Reason: No available tools were provided to assess tool selection criteria
]


🔧 Actual Tool Usage Log:
name='Get Customer ID' description=None reasoning=None output='ID: 1111, Name: Loren' input_parameters=None
name='Get Credit Score' description=None reasoning=None output='455' input_parameters=None
name='Get Account Status' description=None reasoning=None output='Status: good-standing, Nationality: Singaporean' input_parameters=None
name='Get PR Status' description=None reasoning=None output='true' input_parameters=None
name='Risk Policy Search' description=None reasoning=None output='Bank Loan Overall Risk \nThe table below provides the overall risk for initiating a bank loan based on the customer’s \ncredit score and account status. \n \nCredit Score Account Status Overall Risk \n300 – 674 Delinquent High \n675 - 749 Delinquent High \n750 - 850 Delinquent Medium \n300 – 674 Closed High \n675 - 749 Closed Medium \n750 - 850 Closed Low \n300 – 674 Good-standing Medium \n675 - 749 Good-standing Medium \n750 - 850 Good-standing Low\nBank Loan Overall Risk \nThe table below provides the overall risk for initiating a bank loan based on the customer’s \ncredit score and account status. \n \nCredit Score Account Status Overall Risk \n300 – 674 Delinquent High \n675 - 749 Delinquent High \n750 - 850 Delinquent Medium \n300 – 674 Closed High \n675 - 749 Closed Medium \n750 - 850 Closed Low \n300 – 674 Good-standing Medium \n675 - 749 Good-standing Medium \n750 - 850 Good-standing Low\nBank Loan Overall Risk \nThe table below provides the overall risk for initiating a bank loan based on the customer’s \ncredit score and account status. \n \nCredit Score Account Status Overall Risk \n300 – 674 Delinquent High \n675 - 749 Delinquent High \n750 - 850 Delinquent Medium \n300 – 674 Closed High \n675 - 749 Closed Medium \n750 - 850 Closed Low \n300 – 674 Good-standing Medium \n675 - 749 Good-standing Medium \n750 - 850 Good-standing Low' input_parameters=None
name='Interest Rate Policy Search' description=None reasoning=None output='Bank Loan Interest Rate \nThe table below provides the interest rates for initiating a bank loan based on the \ncustomer’s overall risk. \n \nOverall Risk Interest Rate \nLow 3.175 % \nMedium 4.885 % \nHigh 6.325 % \n \nNote: If the risk is between the categories, be conservative in the risk and interest rate.\nBank Loan Interest Rate \nThe table below provides the interest rates for initiating a bank loan based on the \ncustomer’s overall risk. \n \nOverall Risk Interest Rate \nLow 3.175 % \nMedium 4.885 % \nHigh 6.325 % \n \nNote: If the risk is between the categories, be conservative in the risk and interest rate.\nBank Loan Interest Rate \nThe table below provides the interest rates for initiating a bank loan based on the \ncustomer’s overall risk. \n \nOverall Risk Interest Rate \nLow 3.175 % \nMedium 4.885 % \nHigh 6.325 % \n \nNote: If the risk is between the categories, be conservative in the risk and interest rate.' input_parameters=None

---

In [27]:
from deepeval.metrics import (
    ContextualPrecisionMetric, 
    ContextualRecallMetric, 
    ContextualRelevancyMetric
)
from deepeval.test_case import LLMTestCase
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import re

# ==========================================
# 0. HELPER FUNCTIONS
# ==========================================

def extract_claims(text):
    """Split model output into factual claims (sentences)."""
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    return [s for s in sentences if s]


def is_claim_supported(claim, contexts):
    """Naive lexical support check: does claim overlap with any context."""
    claim_tokens = set(claim.lower().split())
    for ctx in contexts:
        ctx_tokens = set(ctx.lower().split())
        if claim_tokens.intersection(ctx_tokens):
            return True
    return False


def is_context_relevant(question, context):
    """
    Simple relevancy heuristic:
    A context is relevant if it shares key tokens with the question.
    """
    q_tokens = set(question.lower().split())
    ctx_tokens = set(context.lower().split())
    return len(q_tokens.intersection(ctx_tokens)) > 0


# ==========================================
# 1. SETUP THE METRICS
# ==========================================

c_precision = ContextualPrecisionMetric(threshold=0.7)
c_recall = ContextualRecallMetric(threshold=0.7)
c_relevancy = ContextualRelevancyMetric(threshold=0.7)

# ==========================================
# 2. DEFINE YOUR TEST SCENARIO
# ==========================================

input_query = "What is the interest rate for Medium risk?"
expected_output = "The interest rate for Medium risk is 4.885%."

prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant. Answer the question based ONLY on the context provided below.

Context:
{context}

Question:
{question}
"""
)

# ==========================================
# 3. GET RETRIEVAL CONTEXT
# ==========================================

search_results = interest_vector_db.similarity_search(input_query, k=3)
retrieval_context = [doc.page_content for doc in search_results]

print("\n📚 Retrieval Context:")
for i, ctx in enumerate(retrieval_context, start=1):
    print(f"{i}. {ctx}\n")

# ==========================================
# 4. RUN RAG CHAIN
# ==========================================

rag_chain = prompt | ChatOpenAI(model="gpt-4o") | StrOutputParser()

actual_output = rag_chain.invoke({
    "context": "\n".join(retrieval_context),
    "question": input_query
})

print(f"\n📝 Generated Output:\n{actual_output}\n")

# ==========================================
# 5. EXTRACT CLAIMS & CHECK SUPPORT
# ==========================================

claims = extract_claims(actual_output)

print("🧾 Extracted Claims:")
for c in claims:
    print("-", c)

supported_claims = []
unsupported_claims = []

for claim in claims:
    if is_claim_supported(claim, retrieval_context):
        supported_claims.append(claim)
    else:
        unsupported_claims.append(claim)

print("\n✅ Supported Claims:")
for c in supported_claims:
    print("-", c)

print("\n❌ Unsupported Claims:")
if unsupported_claims:
    for c in unsupported_claims:
        print("-", c)
else:
    print("None")

# ==========================================
# 6. CONTEXTUAL RELEVANCY (EXPLAINABLE)
# ==========================================

relevant_contexts = []
irrelevant_contexts = []

for ctx in retrieval_context:
    if is_context_relevant(input_query, ctx):
        relevant_contexts.append(ctx)
    else:
        irrelevant_contexts.append(ctx)

relevancy_score_manual = (
    len(relevant_contexts) / len(retrieval_context)
    if retrieval_context else 0.0
)

print("\n🔎 Contextual Relevancy Analysis:")
print(f"Relevant Context Chunks: {len(relevant_contexts)}")
print(f"Irrelevant Context Chunks: {len(irrelevant_contexts)}")
print(f"Manual Contextual Relevancy Score: {relevancy_score_manual:.2f}")

print("\n📌 Relevant Contexts:")
for ctx in relevant_contexts:
    print("-", ctx[:200], "...")

print("\n📌 Irrelevant Contexts:")
for ctx in irrelevant_contexts:
    print("-", ctx[:200], "...")

# ==========================================
# 7. DEEPEVAL TEST CASE
# ==========================================

test_case = LLMTestCase(
    input=input_query,
    actual_output=actual_output,
    expected_output=expected_output,
    retrieval_context=retrieval_context
)

print("\n🚀 Running DeepEval Metrics...\n")

print("🟢 Input Query:")
print(input_query)

print("\n✅ Expected Output:")
print(expected_output)

print("\n📝 Actual Output:")
print(actual_output)

print("\n📚 Retrieval Context:")
for i, ctx in enumerate(retrieval_context, start=1):
    print(f"{i}. {ctx}")

print("\n===================================================\n")

# ==========================================
# 8. RUN METRICS
# ==========================================

c_precision.measure(test_case)
print(f"Contextual Precision: {c_precision.score}")
print(f"Reason: {c_precision.reason}\n")

c_recall.measure(test_case)
print(f"Contextual Recall: {c_recall.score}")
print(f"Reason: {c_recall.reason}\n")

c_relevancy.measure(test_case)
print(f"Contextual Relevancy (DeepEval): {c_relevancy.score}")
print(f"Reason: {c_relevancy.reason}\n")



📚 Retrieval Context:
1. Bank Loan Interest Rate 
The table below provides the interest rates for initiating a bank loan based on the 
customer’s overall risk. 
 
Overall Risk Interest Rate 
Low 3.175 % 
Medium 4.885 % 
High 6.325 % 
 
Note: If the risk is between the categories, be conservative in the risk and interest rate.

2. Bank Loan Interest Rate 
The table below provides the interest rates for initiating a bank loan based on the 
customer’s overall risk. 
 
Overall Risk Interest Rate 
Low 3.175 % 
Medium 4.885 % 
High 6.325 % 
 
Note: If the risk is between the categories, be conservative in the risk and interest rate.

3. Bank Loan Interest Rate 
The table below provides the interest rates for initiating a bank loan based on the 
customer’s overall risk. 
 
Overall Risk Interest Rate 
Low 3.175 % 
Medium 4.885 % 
High 6.325 % 
 
Note: If the risk is between the categories, be conservative in the risk and interest rate.



c:\Users\devil\Downloads\GroupProject\venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


📝 Generated Output:
The interest rate for Medium risk is 4.885%.

🧾 Extracted Claims:
- The interest rate for Medium risk is 4.885%.

✅ Supported Claims:
- The interest rate for Medium risk is 4.885%.

❌ Unsupported Claims:
None

🔎 Contextual Relevancy Analysis:
Relevant Context Chunks: 3
Irrelevant Context Chunks: 0
Manual Contextual Relevancy Score: 1.00

📌 Relevant Contexts:
- Bank Loan Interest Rate 
The table below provides the interest rates for initiating a bank loan based on the 
customer’s overall risk. 
 
Overall Risk Interest Rate 
Low 3.175 % 
Medium 4.885 % 
High  ...
- Bank Loan Interest Rate 
The table below provides the interest rates for initiating a bank loan based on the 
customer’s overall risk. 
 
Overall Risk Interest Rate 
Low 3.175 % 
Medium 4.885 % 
High  ...
- Bank Loan Interest Rate 
The table below provides the interest rates for initiating a bank loan based on the 
customer’s overall risk. 
 
Overall Risk Interest Rate 
Low 3.175 % 
Medium 4.885 % 
High  .

Contextual Precision: 1.0
Reason: The score is 1.00 because all the top-ranked nodes in the retrieval contexts directly provide the table with 'Medium 4.885 %', which exactly answers the question about the interest rate for Medium risk. No irrelevant nodes are ranked above relevant ones—great job!



Contextual Recall: 1.0
Reason: The score is 1.00 because the sentence in the expected output is fully supported by all nodes in the retrieval context, which clearly state the same interest rate for Medium risk.



Contextual Relevancy (DeepEval): 0.6666666666666666
Reason: The score is 0.67 because while the relevant statements like 'Overall Risk Interest Rate: Low 3.175 %, Medium 4.885 %, High 6.325 %' directly answer the input, there are also several irrelevant statements such as 'If the risk is between the categories, be conservative in the risk and interest rate,' which do not pertain to the specific question about Medium risk.



In [ ]:
# 3. GET RETRIEVAL CONTEXT
# ==========================================
# We must fetch exactly what the Agent sees from the Vector DB.
# This assumes 'interest_vector_db' is already defined in your notebook (Cell 3).
search_results = risk_vector_db.similarity_search(input_query2, k=3)
retrieval_context2 = [doc.page_content for doc in search_results]

# Print what was retrieved to verify
print(f"📚 Retrieved {len(retrieval_context2)} chunks from Vector DB.")
# Simple Chain: Prompt -> LLM -> String
rag_chain = prompt | ChatOpenAI(model="gpt-4") | StrOutputParser()

# Run the chain to get the REAL Actual Output
actual_output2 = rag_chain.invoke({
    "context": "\n".join(retrieval_context2), 
    "question": input_query2
})

print(f"📝 Generated Output: {actual_output2}")
# ==========================================
# 4. CREATE TEST CASE & MEASURE
# ==========================================
test_case2 = LLMTestCase(
    input=input_query2,
    actual_output=actual_output2,
    expected_output=expected_output2,
    retrieval_context=retrieval_context2
)

print("\n🚀 Running Evaluation... (This calls the LLM to judge quality)\n")
# 1️⃣ Input query
print(f"🟢 Input Query: {input_query2}\n")

# 2️⃣ Actual output from agent
print(f"📝 Actual Output:\n{actual_output2}\n")

# 3️⃣ Expected output (ground truth)
print(f"✅ Expected Output:\n{expected_output2}\n")

# 4️⃣ Retrieval context (documents from vector DB)
print("📚 Retrieval Context:")
for i, doc in enumerate(retrieval_context2, start=1):
    print(f"{i}. {doc}")


# --- Measure Contextual Precision ---
c_precision.measure(test_case2)
print(f"Contextual Precision: {c_precision.score}")
print(f"Reason: {c_precision.reason}\n")

# --- Measure Contextual Recall ---
c_recall.measure(test_case2)
print(f"Contextual Recall: {c_recall.score}")
print(f"Reason: {c_recall.reason}\n")

# --- Measure Contextual Relevancy ---
c_relevancy.measure(test_case2)
print(f"Contextual Relevancy: {c_relevancy.score}")
print(f"Reason: {c_relevancy.reason}\n")

📚 Retrieved 3 chunks from Vector DB.


c:\Users\devil\Downloads\GroupProject\venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

📝 Generated Output: The risk level for a customer with a closed account status and 700 credit score is Medium.

🚀 Running Evaluation... (This calls the LLM to judge quality)

🟢 Input Query: What is the risk level for customer for account with closed account status and 700 credit score?

📝 Actual Output:
The risk level for a customer with a closed account status and 700 credit score is Medium.

✅ Expected Output:
The risk level for customer with closed account status and 700 credit score is Medium.

📚 Retrieval Context:
1. Bank Loan Overall Risk 
The table below provides the overall risk for initiating a bank loan based on the customer’s 
credit score and account status. 
 
Credit Score Account Status Overall Risk 
300 – 674 Delinquent High 
675 - 749 Delinquent High 
750 - 850 Delinquent Medium 
300 – 674 Closed High 
675 - 749 Closed Medium 
750 - 850 Closed Low 
300 – 674 Good-standing Medium 
675 - 749 Good-standing Medium 
750 - 850 Good-standing Low
2. Bank Loan Overall Risk 
The 

Contextual Precision: 1.0
Reason: The score is 1.00 because all the top-ranked nodes are directly relevant, each providing the exact table and information needed to answer the input question. Great job—no irrelevant nodes ranked above relevant ones!



Contextual Recall: 1.0
Reason: The score is 1.00 because all information in the expected output is fully supported by the tables in all nodes in the retrieval context.



Contextual Relevancy: 0.42857142857142855
Reason: The score is 0.43 because while the context includes a table mapping credit scores and account statuses to risk levels, it does not directly address the specific combination of 'Closed' account status and a 700 credit score, as the relevant range for 'Closed' is 675-749 (Medium risk), but the context does not explicitly mention 700. Additionally, much of the context refers to irrelevant statuses or score ranges, as noted in the irrelevancy reasons.



Loren
<br>
Contextual Precision: 1.0

Reason: The score is 1.00 because all the top-ranked nodes in the retrieval contexts directly provide the information 'Medium 4.885 %', which precisely answers the question about the interest rate for Medium risk. There are no irrelevant nodes ranked above relevant ones, resulting in perfect contextual precision.

Contextual Recall: 1.0

Reason: The score is 1.00 because the sentence in the expected output is fully supported by all nodes in the retrieval context, which clearly state the relevant interest rate.

Contextual Relevancy: 0.75

Reason: The score is 0.75 because while the relevant statements clearly provide the interest rate for Medium risk ('Overall Risk Interest Rate: Low 3.175 %, Medium 4.885 %, High 6.325 %'), there are also some irrelevant statements about risk between categories that do not address the specific input.

---


In [28]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

# ==========================================
# 0. Define expected tool sequence
# ==========================================
expected_tools_sequence = [
    "Get Customer ID",
    "Get Credit Score",
    "Get Account Status",
    "Risk Policy Search",
    "Interest Rate Policy Search"
]

# ==========================================
# 1. Convert tool logs into a readable string
# ==========================================
tool_history_str = "\n".join(
    [f"Step {i+1}: Called tool '{t.name}' with input '{t.input_parameters}'. Output: {t.output}" 
     for i, t in enumerate(tool_usage_log)]
)

loan_process_criteria = """
The agent followed the Loan Processing Workflow logic correctly. 

MANDATORY RULES FOR JUDGE:
1. **Data Efficiency**: 
   - If Nationality is 'Singaporean', the agent is NOT required to check PR Status. (Do not penalize if it does, but reward if it skips it).
   - If Nationality is 'Non-Singaporean', the agent MUST check PR Status.

2. **Policy Adherence (Do not use external banking knowledge)**:
   - **Risk Level 'Medium' IS considered an Approval.** - **Risk Level 'Low' IS considered an Approval.**
   - Only 'High' risk or specific 'Ineligible' status should result in rejection.
   - Do NOT penalize the agent for approving a Medium Risk customer.

3. **Workflow Order**:
   - The agent gathered ID -> Credit Score -> Account Status.
   - It correctly respected the circuit breakers (didn't hallucinate).

4. **Final Output**: 
   - The final decision matches the logic: If Eligible + Low/Medium Risk -> Approve.
"""

# ==========================================
# 2. Create the G-Eval Metric
# ==========================================
workflow_metric = GEval(
    name="Loan Process Adherence",
    criteria=loan_process_criteria,
    evaluation_params=[LLMTestCaseParams.TOOLS_CALLED, LLMTestCaseParams.ACTUAL_OUTPUT], 
    model="gpt-4o"
)

# ==========================================
# 3. Create the Test Case
# ==========================================
test_case = LLMTestCase(
    input=f"Process loan for {customer_name}",
    actual_output=str(crew_output), 
    tools_called=tool_usage_log
)

# ==========================================
# 4. Print Inputs and Tool Info
# ==========================================
print("\n🟢 Input Query / Instruction:")
print(f"Process loan for {customer_name}\n")

print("📝 Actual Output from Agent:")
print(crew_output, "\n")

print("🔧 Tools Called:")
for i, t in enumerate(tool_usage_log, start=1):
    print(f"{i}. Tool Name: {t.name}")
    print(f"   Input Parameters: {t.input_parameters}")
    print(f"   Output: {t.output}\n")

print("📌 Expected Tools Sequence:")
for i, t in enumerate(expected_tools_sequence, start=1):
    print(f"{i}. {t}")

# ==========================================
# 5. Run Workflow Metric
# ==========================================
print("\n🤖 Running G-Eval Judge...")
workflow_metric.measure(test_case)

print(f"\n✅ Workflow Score: {workflow_metric.score}")
print(f"📝 Judge's Reason: {workflow_metric.reason}")


🟢 Input Query / Instruction:
Process loan for loren

📝 Actual Output from Agent:
| Customer ID | Name  | Nationality | PR Status | Credit Score | Risk Level | Interest Rate | Final Decision |
|-------------|-------|-------------|-----------|--------------|------------|---------------|----------------|
| 1111        | Loren | Singaporean | N/A       | 455          | Medium     | 4.885%        | Approved       |

The final decision is to approve the loan for Loren. This decision is based on Loren's eligibility status, credit score, risk level, and the determined interest rate. 

🔧 Tools Called:
1. Tool Name: Get Customer ID
   Input Parameters: None
   Output: ID: 1111, Name: Loren

2. Tool Name: Get Credit Score
   Input Parameters: None
   Output: 455

3. Tool Name: Get Account Status
   Input Parameters: None
   Output: Status: good-standing, Nationality: Singaporean

4. Tool Name: Risk Policy Search
   Input Parameters: None
   Output: Bank Loan Overall Risk 
The table below provide


✅ Workflow Score: 0.8681227203674492
📝 Judge's Reason: The agent correctly skipped the PR Status check for the Singaporean applicant, aligning with data efficiency. The application was approved with a 'Medium' risk level, adhering to policy guidelines. The workflow order was followed correctly: ID, Credit Score, and Account Status. The final decision to approve aligns with the logic for eligible applicants with Low/Medium risk. However, there was a redundant call to the Interest Rate Policy Search, which was unnecessary for the decision-making process.


📝 Judge's Reason: The agent correctly identified the Customer ID and retrieved the Credit Score, as evidenced by the 'Get Customer ID' and 'Get Credit Score' tool calls. However, the risk level determination is incorrect; the credit score of 455 with a 'good-standing' account status should result in a 'Medium' risk level, but this is not clearly stated in the output. The interest rate policy was checked, but the final output does not clearly state the risk level and interest rate, leading to inconsistencies with the data processed.